In [1]:
from __future__ import print_function

import sys

import numpy as np
from pyspark.sql import SparkSession
#查詢最大值最小值
# select 't_coupon_change_log' as tablenm  ,min(id)  as minid,max(id)as maxid, 1 as index  from t_coupon_change_log
# union all
# select 't_coupon_record' as tablenm , min(id) as minid ,max(id) as maxid ,2 as index  from t_coupon_record
# union all
# select 't_coupon_store' as tablenm , min(id) as minid ,max(id) as maxid ,3 as index from t_coupon_store
#############################################################################################################################
# builtins
# __main__
if __name__ == "builtins":
    tableList=[{"tablenm":'t_coupon_change_log', "lowerbound":'1', "upperbound":'513383', "index":'1' },
               {"tablenm":'t_coupon_record', "lowerbound":'1', "upperbound":'2238418', "index":'2' },
               {"tablenm":'t_coupon_store', "lowerbound":'1', "upperbound":'238418', "index":'3'}]
    spark = SparkSession\
        .builder\
        .appName("Python Demo")\
        .enableHiveSupport()\
        .getOrCreate()

    url = "jdbc:postgresql://lovina-prod.ct3lmpqmpvcq.ap-southeast-1.rds.amazonaws.com:5432/coupon"
    user= "elt_sync"
    password="VW7buZwvYWhVMusrkZN1"
    for row in tableList:
        tableNm=row["tablenm"]
        lowerBound=row["lowerbound"]
        upperBound=row["upperbound"]
        df = spark.read.format("jdbc").option("url", url).option("user", user).option("password", password).option("partitionColumn", "id").option("dbtable", tableNm).option("lowerBound", lowerBound).option("upperBound", upperBound).option("numPartitions", 1000).load()
        df.createOrReplaceTempView("t")
        bound = spark.sql("select max(id) max, min(id) min from t")
        max = bound.collect()[0].max
        min = bound.collect()[0].min
        df_table = spark.read.format("jdbc").option("url", url).option("dbtable",  tableNm).option("user", user).option("password", password).option("partitionColumn", "id").option("lowerBound", min).option("upperBound", max).option("numPartitions", 1000).load()
        path="s3://rupiahplus-data-warehouse/aliyun/copuon_emr/"+tableNm
        df_table.write.mode("overwrite").orc(path)
    spark.stop()


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1613975431341_0006,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…